In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv

In [3]:
url = 'https://www.marmiton.org/recettes/recette_tartiflette_10514.aspx'


In [6]:
#effectuer une requête
response = requests.get(url)

# si ok tout va bien :)
if response.ok :
    print('Tout va bien')
else :
    print ("il y a un problème bichette !")

#code 200 = tout va bien !

Tout va bien


In [7]:
# si on veut le contenu sans les balises, très simple il suffit de lui indiquer dans le print

if response.ok :
    soup = BeautifulSoup(response.text, 'lxml')
    title = soup.find('title') #on met juste le nom de la balise
    print(title.text)


Tartiflette : recette de Tartiflette


In [6]:
#recherche de la liste des ingrédients

liste_ingredients = []


if response.ok :
    soup = BeautifulSoup(response.text, 'html.parser')

    for i in soup.findAll('span', {'class' : 'SHRD__sc-10plygc-0 kWuxfa'}) :
        try :
            liste_ingredients.append((i.find('span',{'class' : "RCP__sc-8cqrvd-3 itCXhd"})).text) 
            

        except AttributeError:
            liste_ingredients.append((i.find('span',{'class' : "RCP__sc-8cqrvd-3 cDbUWZ"})).text) 
            pass
    
    print(liste_ingredients) 


['reblochon', 'pomme de terre', 'oignon', 'sel', 'poivre', 'jambon fumé', 'Apremont']


In [7]:
#recherche des quantités

quantity = []

if response.ok :
    soup = BeautifulSoup(response.text, 'html.parser')

    for i in soup.findAll('div', {'class' : 'RCP__sc-8cqrvd-0 hSorOY'}) :
        quantite = i.find('span',{'class' : "SHRD__sc-10plygc-0 epviYI"}).text.replace('\xa0',' ')
        
        quantity.append(quantite) 
            
   
print(quantity)

['1.5', '1 kg', '500 g', '1 pincée', '1 pincée', '1 tranche', '40 cl']


In [8]:
#create dictionnary with 2 lists

value = liste_ingredients
ingredients_complets = dict(zip(value, quantity))

ingredients_complets

{'reblochon': '1.5',
 'pomme de terre': '1 kg',
 'oignon': '500 g',
 'sel': '1 pincée',
 'poivre': '1 pincée',
 'jambon fumé': '1 tranche',
 'Apremont': '40 cl'}

__Plus simple, création d'un dictionnaire avec ingredient + valeur en une seule fois__


In [14]:
#plus simple, création d'un dictionnaire avec ingredient + valeur

liste_ingredients = []
quantity = []


if response.ok :
    soup = BeautifulSoup(response.text, 'html.parser')

    for i in soup.findAll('span', {'class' : 'SHRD__sc-10plygc-0 kWuxfa'}) :
        try :
            liste_ingredients.append((i.find('span',{'class' : "RCP__sc-8cqrvd-3 itCXhd"})).text) 
            

        except AttributeError:
            liste_ingredients.append((i.find('span',{'class' : "RCP__sc-8cqrvd-3 cDbUWZ"})).text) 
            pass
    
    for i in soup.findAll('div', {'class' : 'RCP__sc-8cqrvd-0 hSorOY'}) :
        quantite = i.find('span',{'class' : "SHRD__sc-10plygc-0 epviYI"}).text.replace('\xa0',' ')
        quantity.append(quantite) 
    
    

ingredients_complets = dict(zip(liste_ingredients, quantity))

ingredients_complets  

{'reblochon': '1.5',
 'pomme de terre': '1 kg',
 'oignon': '500 g',
 'sel': '1 pincée',
 'poivre': '1 pincée',
 'jambon fumé': '1 tranche',
 'Apremont': '40 cl'}

__Création d'un df que nous exporterons ensuite dans un excel ou csv__

In [68]:
df = pd.DataFrame.from_dict(ingredients_complets, orient='index')                           #creation du df
df = df.reset_index(drop=False).rename(columns={"index": "Ingrédients", 0: "Quantité"})     #reset index, rename colonne
df[['Quantité','Unité']] = df['Quantité'].str.split(expand=True)                            #split des quantités pour avoir les unités à part
df['Unité'] = df['Unité'].replace({None :'pièce'})                                          # remplacement des valeurs 'none' par 'pièce'
df


,Ingrédients,Quantité,Unité
0,reblochon,1.5,pièce
1,pomme de terre,1,kg
2,oignon,500,g
3,sel,1,pincée
4,poivre,1,pincée
5,jambon fumé,1,tranche
6,Apremont,40,cl


#### <span style="color: #99CCFF"> **Création d'un fichier csv**</span>

In [81]:
df.to_csv('table_de_la_recette.csv',columns=['Ingrédients', 'Quantité','Unité'],index=False)
